In [1]:
import pandas as pd
import openai
import json
from collections import Counter
import os
from dotenv import load_dotenv
import re
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type
)  # for exponential backoff


In [2]:
#Uploading the experiment data used (update path to data)
packages_df = pd.read_csv('upworthy_experiment_packages.csv', low_memory=False)
packages_df = packages_df.drop(columns=['Unnamed: 0', 'Unnamed: 1'])

In [3]:
packages_df.head()

,created_at,updated_at,clickability_test_id,excerpt,headline,lede,slug,eyecatcher_id,impressions,clicks,significance,first_place,winner,share_text,square,test_week,created
0,2014-11-20 11:33:26.475,2016-04-02 16:25:54.046,546dd17e26714c82cc00001c,Things that matter. Pass 'em on.,"Let’s See … Hire Cops, Pay Teachers, Buy Books...",<p>Iff you start with the basic fact that inno...,let-s-see-hire-cops-pay-teachers-buy-books-for...,546dce659ad54ec65b000041,3118,8,0.1,False,False,NaN,NaN,201446,2014-11-20 11:33:26.475
1,2014-11-20 15:00:01.032,2016-04-02 16:25:54.128,546e01d626714c6c4400004e,Things that matter. Pass 'em on.,People Sent This Lesbian Questions And Her Rai...,<p>I'll be honest. I've wondered about 7.</p>,people-sent-this-lesbian-questions-and-her-rai...,546d1b4bfd3617f091000041,4587,130,55.8,False,False,NaN,NaN,201446,2014-11-20 15:00:01.032
2,2014-11-20 11:33:51.973,2016-04-02 16:25:54.069,546dd17e26714c82cc00001c,Things that matter. Pass 'em on.,$3 Million Is What It Takes For A State To Leg...,<p>Iff you start with the basic fact that inno...,3-million-is-what-it-takes-for-a-state-to-lega...,546dce659ad54ec65b000041,3017,19,26.9,False,False,NaN,NaN,201446,2014-11-20 11:33:51.973
3,2014-11-20 11:34:12.107,2016-04-02 16:25:54.049,546dd17e26714c82cc00001c,Things that matter. Pass 'em on.,The Fact That Sometimes Innocent People Are Ex...,<p>Iff you start with the basic fact that inno...,the-fact-that-sometimes-innocent-people-are-ex...,546dce659ad54ec65b000041,2974,26,100.0,True,False,NaN,NaN,201446,2014-11-20 11:34:12.107
4,2014-11-20 11:34:33.935,2016-04-02 16:25:54.072,546dd17e26714c82cc00001c,Things that matter. Pass 'em on.,Reason #351 To End The Death Penalty: It Costs...,<p>Iff you start with the basic fact that inno...,reason-351-to-end-the-death-penalty-it-costs-3...,546dce659ad54ec65b000041,3050,10,0.2,False,False,NaN,NaN,201446,2014-11-20 11:34:33.935


In [4]:

load_dotenv()

# Set OpenAI API key
openai.api_key = os.environ['OPENAI_API_KEY']

def ChatGPT_request(prompt, temp, model="gpt-3.5"):
  """
  Given a prompt and a dictionary of GPT parameters, make a request to OpenAI
  server and returns the response.
  ARGS:
    prompt: a str prompt
  RETURNS:
    a str of GPT-3's response.
  """

  if model == "gpt-3.5":
    model = "gpt-3.5-turbo"
  elif model == "gpt-4":
    model = "gpt-4"
  else:
    model = "gpt-3.5-turbo"
  # temp_sleep()
  try:
    completion = openai.ChatCompletion.create(
    model=model,
    temperature=temp,
    messages=[
        {
            "role": "user",
            "content": prompt
        }
      ]
    )
    return completion["choices"][0]["message"]["content"]

  except:
    print ("ChatGPT ERROR")
    return "ChatGPT ERROR"


def persona_ChatGPT_request(prompt, persona_system_message):
  """
  Given a prompt and a dictionary of GPT parameters, make a request to OpenAI
  server and returns the response.
  ARGS:
    prompt: a str prompt
    persona_system_message: a str of persona system message
  RETURNS:
    a str of GPT-3's response.
  """
  # temp_sleep()
  try:
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": persona_system_message
        },
        {
            "role": "user",
            "content": prompt
        }
      ]
    )
    return completion["choices"][0]["message"]["content"]

  except:
    print ("ChatGPT ERROR")
    return "ChatGPT ERROR"


def ChatGPT_safe_generate_response(prompt,
                                   example_output,
                                   special_instruction,
                                   repeat=3,
                                   fail_safe_response="error",
                                   func_validate=None,
                                   func_clean_up=None,
                                   verbose=False):
  # prompt = 'GPT-3 Prompt:\n"""\n' + prompt + '\n"""\n'
  prompt = '"""\n' + prompt + '\n"""\n'
  prompt += f"Output the response to the prompt above in json. {special_instruction}\n"
  prompt += "Example output json:\n"
  prompt += '{"output": "' + str(example_output) + '"}'

  if verbose:
    print ("CHAT GPT PROMPT")
    print (prompt)

  for i in range(repeat):

    try:
      curr_gpt_response = ChatGPT_request(prompt).strip()
      end_index = curr_gpt_response.rfind('}') + 1
      curr_gpt_response = curr_gpt_response[:end_index]
      curr_gpt_response = json.loads(curr_gpt_response)["output"]

      if func_validate(curr_gpt_response, prompt=prompt):
        return func_clean_up(curr_gpt_response, prompt=prompt)

      if verbose:
        print ("---- repeat count: \n", i, curr_gpt_response)
        print (curr_gpt_response)
        print ("~~~~")

    except:
      pass

  return False

def format_paragraph(paragraph):
    sentences = re.split(r'(?<=[.!?])\s+', paragraph)
    formatted_paragraph = '\n'.join(sentences)
    # wrap with triple quotes
    return formatted_paragraph

## Generate preferences for average - views/justification - response 

In [6]:
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory
from langchain.callbacks import get_openai_callback

llm = OpenAI(temperature=0, OPENAI_API_KEY=openai.api_key, 
model_name='gpt-3.5-turbo')
template = """You are a subscriber of news website upworthy given the following headline and your initial\
views of it. 
You should  account for your preferences as well as if you find the wording of the headline compelling or not
Return response as 0 if no, and 1 if you would read it. Every now and then you might read things outside of your preferences too
Headline: {headline}
Preferences: {preferences}
Views: {views}

Response: [<integer>]
"""
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=0.7,model_name='gpt-3.5-turbo')
views_template = """You are a subscriber of news website upworthy, given the following headline, articulate
your views around the headline based on your preferences and if you find it interesting or not and be very critical of what you think keep it to less than 250 characters.

Headline: {headline}
Prefernes: {preferences}
Views:"""
views_prompt_template = PromptTemplate(input_variables=["headline", "preferences"], template=views_template)
views_chain = LLMChain(llm=llm, prompt=views_prompt_template, output_key="views")

preferences_template = """You are  a subscriber of news website upworthy considering that define a list of preferences
for articles you would be really interested in reading - keep it to your top 5 interests.

Preferences:"""
preferences_prompt_template = PromptTemplate(input_variables = [],template=preferences_template)
preferences_chain = LLMChain(llm=llm, prompt=preferences_prompt_template, output_key="preferences")

prompt_template = PromptTemplate(input_variables=["headline", "views","preferences"], template=template)
reasoning_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="response")

overall_chain = SequentialChain(
    chains=[preferences_chain, views_chain, reasoning_chain],
    input_variables=["headline"],
    # Here we return multiple variables
    output_variables=["response"],
    verbose=True)

/export/home/dor/akamal/.local/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/export/home/dor/akamal/.local/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## Data Cleaning and Filtering

In [7]:
# Find duplicate headlines (some headlines are duplicated and need to be deleted)
duplicates = packages_df[packages_df.duplicated('headline', keep=False)]

# Extract unique clickability_test_id corresponding to duplicate headlines
drop_ids = duplicates['clickability_test_id'].unique()

# Drop these IDs from the original DataFrame
packages_df_2 = packages_df[~packages_df['clickability_test_id'].isin(drop_ids)]

# Display the filtered DataFrame
print("\nFiltered DataFrame:")
print(packages_df_2.shape)



Filtered DataFrame:
(9922, 17)


In [8]:
# ID Filter for packages which have at least 2 unique headlines
valid_ids = packages_df_2.groupby('clickability_test_id').filter(lambda x: x['headline'].nunique() >= 2)['clickability_test_id'].unique()

# Sample 50 unique clickability_test_id -> use for a new run
sample_packages = 250
sampled_ids = pd.Series(valid_ids).sample(sample_packages, random_state=123)

# Recalling from current results
# sampled_ids = sample_ids_latest

# Fetch all rows associated with these sampled IDs
sampled_df = packages_df_2[packages_df_2['clickability_test_id'].isin(sampled_ids)]


## Asynch calls

In [10]:
import asyncio
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



async def async_generate(chain, headline):
    resp = await chain.arun(headline=headline)
    return resp


async def generate_concurrently(samples, headline):
    tasks = [async_generate(overall_chain, headline) for _ in range(samples)]
    results = await asyncio.gather(*tasks)
    return results 




In [20]:
results = []
i=0
# Define number of personas
num_personas = 50
from datetime import datetime
import time
for _, row in sampled_df.iterrows():   
    init_time = datetime.now()
    responses = await generate_concurrently(samples=num_personas, headline=row['headline'])
    finish_time = datetime.now()
    final_time = finish_time-init_time
    print(i)
    print('Time:'+ str(final_time.total_seconds()))
    results.append({
        'clickability_test_id': row['clickability_test_id'],
        'headline': row['headline'],
        'response': responses
    })
    i+=1
    print(i)




> Entering new SequentialChain chain...

> Entering new SequentialChain chain...


> Entering new SequentialChain chain...



> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain c

Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.
0
Time:12.611291
1


> Entering new SequentialChai

Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.
1
Time:10.836257
2


> Entering new SequentialChai

...


> Finished chain.


Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} <CIMultiDictProxy('Date': 'Fri, 06 Oct 2023 17:02:01 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '811f69971d89ff94-BOS', 'alt-svc': 'h3=":443"; ma=86400')>.



> Finished chain.


Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} <CIMultiDictProxy('Date': 'Fri, 06 Oct 2023 17:02:20 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '811f6a0a58ca3b69-BOS', 'alt-svc': 'h3=":443"; ma=86400')>.



> Finished chain.


In [13]:
results_df = pd.DataFrame(results)
results_df['response'] = results_df['response'].apply(lambda x: [int(i) for i in x])
results_df['response_synthetic'] = results_df['response'].apply(lambda x: 100*sum(x)/len(x))
# Generate response rates for synthetic personas
results_df_v2 = results_df.drop(columns=['response'])

In [14]:
# Generate response rates for original data
filter_cols = ['clickability_test_id', 'headline','impressions', 'clicks']
sampled_df_2 = sampled_df[filter_cols]
sampled_df_2['clicks']= sampled_df_2['clicks'].astype('int')
sampled_df_2['impressions']= sampled_df_2['impressions'].astype('int')
sampled_df_2['response_original'] = 100*(sampled_df_2['clicks']/sampled_df_2['impressions'])

/tmp/ipykernel_273877/553319166.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df_2['clicks']= sampled_df_2['clicks'].astype('int')
/tmp/ipykernel_273877/553319166.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df_2['impressions']= sampled_df_2['impressions'].astype('int')
/tmp/ipykernel_273877/553319166.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [15]:
# Merge results for synthetic and original data
merged_results = results_df_v2.merge(sampled_df_2, on=['clickability_test_id', 'headline'],
                                       how='inner')

In [16]:
merged_results.sort_values(['clickability_test_id', 'headline'])

,clickability_test_id,headline,response_synthetic,impressions,clicks,response_original
0,54903d5833353100214f0000,"They're Not Just Working Out, They're Also Pro...",78.0,2788,27,0.968436
2,5497ab24363335001b250000,Exquisite Recordings Let You Listen In On Mome...,58.0,3031,13,0.428901
1,5497ab24363335001b250000,Magical Sounds Make The Earth Seem Smaller. Li...,22.0,3020,7,0.231788


In [ ]:
# Save results

from datetime import datetime

# Save data
current_time = datetime.now()
# Format the date and time
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
folder_path = f"results/{formatted_time}"
os.mkdir(folder_path)

# Save results
merged_results.to_csv(f"results/{formatted_time}/merged_results.csv")
results_df_v2.to_csv(f"results/{formatted_time}/raw_responses.csv")
sampled_df.to_csv(f"results/{formatted_time}/sampled_df.csv")


## Analyse results

In [ ]:
# Rank headlines for each clickablity_test_id

In [ ]:
merged_results

In [ ]:
analysis_df = merged_results.copy()

# Rank the headlines based on response rate for both sets of responses within each clickability_test_id
analysis_df['rank_original'] = analysis_df.groupby('clickability_test_id')['response_original'].rank(ascending=False, method='min')
analysis_df['rank_synthetic'] = analysis_df.groupby('clickability_test_id')['response_synthetic'].rank(ascending=False, method='min')

# Display the updated data with ranks
analysis_df[['clickability_test_id', 'headline', 'rank_original', 'rank_synthetic']].head()
analysis_df['top_match'] = (analysis_df['rank_original'] == 1) & (analysis_df['rank_synthetic'] == 1)


In [ ]:
from sklearn.metrics import roc_auc_score

# Create binary labels: 1 for top-ranked headlines based on original response, 0 otherwise
y_true = (analysis_df['rank_original'] == 1).astype(int)

# Use ranks based on synthetic response as predicted probabilities. Invert ranks so that higher rank has higher value.
y_score = 1 / analysis_df['rank_synthetic']

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_true, y_score)
roc_auc

Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ziOHzvweXIt2CxdP4bGi94yg on tokens per min. Limit: 90000 / min. Current: 89746 / min. Contact us through our help center at help.openai.com if you continue to have issues..



> Finished chain.

> Finished chain.


Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ziOHzvweXIt2CxdP4bGi94yg on tokens per min. Limit: 90000 / min. Current: 89889 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ziOHzvweXIt2CxdP4bGi94yg on tokens per min. Limit: 90000 / min. Current: 89886 / min. Contact us through our help center at help.openai.com if you continue to have issues..



> Finished chain.

> Finished chain.


Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ziOHzvweXIt2CxdP4bGi94yg on tokens per min. Limit: 90000 / min. Current: 89397 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ziOHzvweXIt2CxdP4bGi94yg on tokens per min. Limit: 90000 / min. Current: 89645 / min. Contact us through our help center at help.openai.com if you continue to have issues..



> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.
